# 自作のPPOコード（環境並列版）

In [9]:
from dataclasses import dataclass, field, asdict, is_dataclass

import sys
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal

import gymnasium as gym

from myActivator import tanhAndScale
from myFunction import make_squashed_gaussian

In [10]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    stream=sys.stdout, datefmt="%H:%M:%S")

In [11]:
# 並列環境数
num_envs = 4  # CPUコア数やタスクの重さに応じて調整

# --- 1. スペック確認用 (単一環境) ---
# VectorEnv越しだと内部パラメータ(m, l, gなど)が見にくいので、
# 確認用に一時的に単一環境を作ってログを出力します。
temp_env = gym.make("Pendulum-v1")
logging.info("=== Environment Specs (Reference) ===")
for key in vars(temp_env.spec):
    logging.info('%s: %s', key, vars(temp_env.spec)[key])
for key in vars(temp_env.unwrapped):
    # すべて表示すると多すぎる場合は必要なものだけでもOK
    logging.info('%s: %s', key, vars(temp_env.unwrapped)[key])
temp_env.close()

# --- 2. 学習用環境の立ち上げ (並列化) ---
# make_vec を使うのが現代的なGymnasiumの書き方です。
# vectorization_mode="async": マルチプロセス (重い環境向け)
# vectorization_mode="sync": シングルプロセス (Pendulumのような軽い環境ならこれでも十分高速)
env = gym.make_vec("Pendulum-v1", num_envs=num_envs, vectorization_mode="async")

logging.info(f"=== VectorEnv Created: {num_envs} parallel environments ===")

01:48:39 [INFO] === Environment Specs (Reference) ===
01:48:39 [INFO] id: Pendulum-v1
01:48:39 [INFO] entry_point: gymnasium.envs.classic_control.pendulum:PendulumEnv
01:48:39 [INFO] reward_threshold: None
01:48:39 [INFO] nondeterministic: False
01:48:39 [INFO] max_episode_steps: 200
01:48:39 [INFO] order_enforce: True
01:48:39 [INFO] disable_env_checker: False
01:48:39 [INFO] kwargs: {}
01:48:39 [INFO] additional_wrappers: ()
01:48:39 [INFO] vector_entry_point: None
01:48:39 [INFO] namespace: None
01:48:39 [INFO] name: Pendulum
01:48:39 [INFO] version: 1
01:48:39 [INFO] max_speed: 8
01:48:39 [INFO] max_torque: 2.0
01:48:39 [INFO] dt: 0.05
01:48:39 [INFO] g: 10.0
01:48:39 [INFO] m: 1.0
01:48:39 [INFO] l: 1.0
01:48:39 [INFO] render_mode: None
01:48:39 [INFO] screen_dim: 500
01:48:39 [INFO] screen: None
01:48:39 [INFO] clock: None
01:48:39 [INFO] isopen: True
01:48:39 [INFO] action_space: Box(-2.0, 2.0, (1,), float32)
01:48:39 [INFO] observation_space: Box([-1. -1. -8.], [1. 1. 8.], (3,)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
@dataclass
class Config:
    # ===== Pendulum-v1 specs =====
    obs_dim: int = 3
    act_dim: int = 1

    # TRPOAgent 側は Config.u_llim / Config.u_ulim を参照
    u_llim: list[float] = field(default_factory=lambda: [-2.0])
    u_ulim: list[float] = field(default_factory=lambda: [ 2.0])

    # ===== Network architecture =====
    V_net_in: int = 3
    P_net_in: int = 3

    V_net_sizes: list[int] = field(default_factory=lambda: [64, 64])
    P_net_sizes: list[int] = field(default_factory=lambda: [64, 64])

    V_net_out: int = 1
    P_net_out: int = 1  # = act_dim

    # ===== Optimizer =====
    V_lr: float = 1e-3
    P_lr: float = 3e-4

    # ===== GAE / discount =====
    gamma: float = 0.99
    lam: float = 0.97

    # ===== PPO hyperparameters =====
    clip_ratio: float = 0.2
    policy_train_iters: int = 50
    target_kl: float = 0.01
    reward_scaling: float = 0.01

    # ===== Value function training =====
    value_train_iters: int = 50
    value_l2_reg: float = 1e-3
    v_clip_epsilon: float = 0.2

In [14]:
from myAgent import PPOAgent

In [15]:
import numpy as np
import torch
import logging

def train_ppo_parallel(
    env,
    agent,
    total_step: int = 2000, # ループを回す回数（フレーム数ではない点に注意）
    batch_steps: int = 128,  # PPOの1回の更新で回すタイムステップ数 (T)
    random_steps: int = 0,
    bootstrap_on_timeout: bool = False,
    log_interval_updates: int = 1,
    eval_env = None, # 評価用には別途「単一環境」を渡すのがベスト
):
    """
    並列環境 (VectorEnv) 専用のPPO学習ループ
    """
    print(f"Start PPO Parallel Training: Device={agent.device}, Steps={batch_steps}, Envs={env.num_envs}")

    # 変数名修正に伴う変更
    low_np = agent.u_low.detach().cpu().numpy()
    high_np = agent.u_high.detach().cpu().numpy()

    # 履歴保存用
    loss_history = {"policy_loss": [], "value_loss": []}
    train_reward_history = [] 

    # ★変更点1: 現在のスコア保存用変数を「配列」にする
    # 各環境ごとの現在の報酬積み上げ
    current_ep_returns = np.zeros(env.num_envs, dtype=np.float32)

    # rollout buffer
    rollout = {"obs": [], "act": [], "logp": [], "rew": [], "obs_next": [], "done": []}

    def rollout_clear():
        for k in rollout:
            rollout[k].clear()

    # ★変更点2: Resetは最初の一回だけ
    obs, info = env.reset()
    # obs は (N, obs_dim) の形

    update_num = 0

    # total_steps は「全環境合計のステップ数」ではなく「ループ回数」として扱います
    # 実際の消化フレーム数は total_steps * num_envs になります
    for t in range(total_step):
        
        # --- (1) 行動選択 ---
        # 並列環境なので、obs は (N, dim) です。
        # Agent側は (N, dim) を受け取って (N, act_dim) を返すようになっています。
        if t < random_steps:
            # ランダム
            # vector env の sample は (N, act_dim) を返すはずですが、念のため確認推奨
            # ここでは安全策で agent を通してサンプリングします
            with torch.no_grad():
                 a_t, logp_t = agent.get_action_and_log_prob(obs, deterministic=False)
                 logp_val = logp_t.cpu().numpy() # (N,)
        else:
            with torch.no_grad():
                a_t, logp_t = agent.get_action_and_log_prob(obs, deterministic=False)
            
            # Tensor -> Numpy
            action_raw = a_t.detach().cpu().numpy() # (N, act_dim)
            logp_val = logp_t.detach().cpu().numpy() # (N,)

        # --- (2) env step ---
        # 行動のクリップ (N, act_dim) に対して一括で行われます
        action_env = np.clip(action_raw, low_np, high_np)
        
        # VectorEnv は step すると以下が返ってきます (全て配列)
        # obs_next: (N, obs_dim)
        # reward: (N,)
        # terminated: (N,) bool
        # truncated: (N,) bool
        obs_next, reward, terminated, truncated, info = env.step(action_env)

        # ★変更点3: 報酬の積み上げ処理
        # ベクトル同士の足し算 (N,) += (N,)
        current_ep_returns += reward

        # GAE計算用のdoneフラグ
        if bootstrap_on_timeout:
            done_for_gae = terminated
        else:
            done_for_gae = np.logical_or(terminated, truncated)
        
        # AI学習用報酬のスケーリング (N,)
        scaled_reward = reward * agent.Config.reward_scaling

        # --- (3) バッファに保存 ---
        # 全て (N, ...) の形のままリストに追加します
        rollout["obs"].append(obs)           # (N, dim)
        rollout["act"].append(action_raw)    # (N, act)
        rollout["logp"].append(logp_val)     # (N,)
        rollout["rew"].append(scaled_reward) # (N,)
        rollout["obs_next"].append(obs_next) # (N, dim)
        rollout["done"].append(done_for_gae.astype(np.float32)) # (N,)

        # ★変更点4: 終了判定とリセット処理 (Auto-Reset対応)
        # VectorEnvは「終了した環境だけ内部で勝手にリセット」され、
        # obs_next には「新しいエピソードの初期状態」が入って返ってきます。
        # なので手動の env.reset() は削除します。

        # どこの環境が終わったかチェックしてログを残す
        dones = np.logical_or(terminated, truncated) # (N,)
        
        # 終了した環境があれば、そのスコアを履歴に移して、カウンタを0に戻す
        if dones.any():
            for i, is_done in enumerate(dones):
                if is_done:
                    train_reward_history.append(current_ep_returns[i])
                    current_ep_returns[i] = 0.0
                    
                    # 厳密なPPO実装メモ:
                    # obs_next[i] はリセット後の初期状態になっています。
                    # 正確な学習のためには「リセット前の到達地点(terminal state)」を使うべき場合があります。
                    # Gymnasiumでは info["final_observation"][i] にそれが入っています。
                    # 今回はコードを複雑にしないため、このまま obs_next を使います。

        # 次の状態へ更新
        obs = obs_next

        # --- (5) Update ---
        # 指定ステップ数 (batch_steps) だけデータが溜まったら更新
        if len(rollout["obs"]) >= batch_steps:
            update_num += 1

            # list -> numpy stacking
            # 結果は (T, N, dim) になります
            states      = np.stack(rollout["obs"], axis=0)
            actions     = np.stack(rollout["act"], axis=0)
            log_probs   = np.stack(rollout["logp"], axis=0)
            rewards     = np.stack(rollout["rew"], axis=0)
            states_next = np.stack(rollout["obs_next"], axis=0)
            dones       = np.stack(rollout["done"], axis=0)

            # Update実行
            # Agent側で (T, N, dim) を受け取れるように修正したのでそのまま渡す
            loss_dict = agent.update_net(states, actions, log_probs, rewards, states_next, dones)
            
            rollout_clear()

            loss_history["policy_loss"].append(loss_dict["policy_loss"])
            loss_history["value_loss"].append(loss_dict["value_loss"])

            if (update_num % log_interval_updates) == 0:
                # 評価: 並列環境(VectorEnv)で evaluate を回すとバグる（終わらない）ので
                # 評価用に別途「単一環境(eval_env)」を渡してある場合のみ実行するように変更
                eval_score = 0.0
                if eval_env is not None:
                    eval_score = evaluate(eval_env, agent, n_episodes=3)
                else:
                    # 評価環境がない場合は、学習中の平均報酬を表示しておく
                    if len(train_reward_history) > 0:
                        eval_score = np.mean(train_reward_history[-10:])
                
                logging.info(
                    f"Update {update_num:4d} | Step {t:6d} | "
                    f"Eval/MeanReward: {eval_score:8.2f} | "
                    f"P_Loss: {loss_dict['policy_loss']:.4f} | "
                    f"V_Loss: {loss_dict['value_loss']:.4f}"
                )

    return loss_history, train_reward_history


def evaluate(env, agent, n_episodes=3):
    """
    並列環境 (VectorEnv) 専用の評価関数
    指定した n_episodes 分のエピソードが完了するまで全環境を回し続け、平均スコアを返します。
    """
    # 行動の制限値
    low_np = agent.u_low.detach().cpu().numpy()
    high_np = agent.u_high.detach().cpu().numpy()

    # 結果保存用リスト
    episode_scores = []
    
    # 各環境の現在の報酬を保持する配列 (サイズ: num_envs)
    current_rewards = np.zeros(env.num_envs, dtype=np.float32)
    
    # 完了したエピソード数をカウント
    finished_count = 0
    
    obs, _ = env.reset()

    # 指定数のエピソードが集まるまでループ
    while finished_count < n_episodes:
        # Agentは (N, dim) を受け取り (N, act) を返す
        # 評価時は決定論的(deterministic=True)に振る舞う
        action = agent.step(obs)
        action = np.clip(action, low_np, high_np)
        
        # Step (VectorEnvなので全環境が一斉に進む)
        obs, reward, terminated, truncated, _ = env.step(action)
        
        # 報酬を加算 (ベクトル演算)
        current_rewards += reward
        
        # 終了判定
        dones = np.logical_or(terminated, truncated)
        
        # 終了した環境があればスコアを回収
        if dones.any():
            for i, is_done in enumerate(dones):
                if is_done:
                    # まだ必要数に達していなければ記録
                    if finished_count < n_episodes:
                        episode_scores.append(current_rewards[i])
                        finished_count += 1
                    
                    # カウンタリセット (次のエピソードの準備)
                    current_rewards[i] = 0.0

    return np.mean(episode_scores)

In [16]:
agent = PPOAgent(Config=Config(),device=device)
total_step= 100000

lh, rh = train_ppo_parallel(
    env=env,
    agent=agent,
    total_step=total_step,
    batch_steps=2048,
)

Start PPO Parallel Training: Device=cuda, Steps=2048, Envs=4
01:48:45 [INFO] Increased policy learning rate to 0.00045
01:48:46 [INFO] Update    1 | Step   2047 | Eval/MeanReward: -1132.15 | P_Loss: -0.0025 | V_Loss: 1.4181


/home/hirokazumura/benkyo_kojin/rl_python/myAgent.py:1113: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/aten/src/ATen/native/Scalar.cpp:22.)
  return {"policy_loss": policy_loss.item(), "value_loss": value_loss.item()}


01:48:51 [INFO] Increased policy learning rate to 0.000675
01:48:51 [INFO] Update    2 | Step   4095 | Eval/MeanReward: -1170.33 | P_Loss: -0.0045 | V_Loss: 1.4008
01:48:54 [INFO] Increased policy learning rate to 0.0010125
01:48:54 [INFO] Update    3 | Step   6143 | Eval/MeanReward: -1133.58 | P_Loss: -0.0033 | V_Loss: 1.2602
01:49:00 [INFO] Increased policy learning rate to 0.00151875
01:49:00 [INFO] Update    4 | Step   8191 | Eval/MeanReward: -1171.38 | P_Loss: -0.0037 | V_Loss: 1.1278
01:49:05 [INFO] Increased policy learning rate to 0.0022781249999999998
01:49:05 [INFO] Update    5 | Step  10239 | Eval/MeanReward: -1223.30 | P_Loss: -0.0041 | V_Loss: 0.8637
01:49:11 [INFO] Increased policy learning rate to 0.0034171874999999997
01:49:11 [INFO] Update    6 | Step  12287 | Eval/MeanReward: -1226.90 | P_Loss: -0.0061 | V_Loss: 0.8263
01:49:16 [INFO] Increased policy learning rate to 0.005125781249999999
01:49:16 [INFO] Update    7 | Step  14335 | Eval/MeanReward: -1125.10 | P_Loss: 

In [17]:
env.close()

In [18]:
from pathlib import Path
from datetime import datetime

def make_unique_path(path: str | Path) -> Path:
    """
    path が既に存在する場合、末尾に _1, _2, ... を付けて未使用のパスを返す。
    例: ddpg_final_20251221_235959.pth -> ddpg_final_20251221_235959_1.pth -> ...
    """
    p = Path(path)

    # 存在しないならそのまま使う
    if not p.exists():
        return p

    parent = p.parent
    stem = p.stem      # 拡張子抜きファイル名
    suffix = p.suffix  # ".pth"

    i = 1
    while True:
        cand = parent / f"{stem}_{i}{suffix}"
        if not cand.exists():
            return cand
        i += 1


# 推論用に eval モードにしておく（保存自体は train のままでも可）
agent.mode2eval()

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")

models_dir = Path("./models")
models_dir.mkdir(parents=True, exist_ok=True)

base_path = models_dir / f"ppo_final_{stamp}.pth"
save_path = make_unique_path(base_path)

agent.save_all(
    save_path.as_posix(),
    extra={
        "total_step": int(total_step),
        "reward_history": rh,  # 必要ならそのままでOK
    }
)

print(f"saved to {save_path}")

saved to models/ppo_final_20260206_015253.pth
